<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Train-cats-for-the-DFs" data-toc-modified-id="Train-cats-for-the-DFs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train cats for the DFs</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#proc_df-to-fill-nas,-one-hot-encode-cats" data-toc-modified-id="proc_df-to-fill-nas,-one-hot-encode-cats-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>proc_df to fill nas, one hot encode cats</a></span></li><li><span><a href="#Mark-as-categorical-variables-(not-required)" data-toc-modified-id="Mark-as-categorical-variables-(not-required)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Mark as categorical variables (not required)</a></span></li><li><span><a href="#One-hot-encoding-categories-(not-required)" data-toc-modified-id="One-hot-encoding-categories-(not-required)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>One hot encoding categories (not required)</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Add-exta-interpreted-columns-on-aggregates" data-toc-modified-id="Add-exta-interpreted-columns-on-aggregates-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Add exta interpreted columns on aggregates</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Add date part

In [6]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Train cats for the DFs

In [7]:
train_cats(hist_trans)
train_cats(new_hist_trans)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [8]:
dfs = [hist_trans, new_hist_trans]

In [9]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    return df

In [10]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [11]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### proc_df to fill nas, one hot encode cats

In [12]:
%time hist_trans_proc, y_hist, nas = proc_df(hist_trans, max_n_cat=7,ignore_flds=['purchase_date','card_id', 'merchant_id'])

CPU times: user 48.6 s, sys: 9.59 s, total: 58.2 s
Wall time: 12.8 s


In [13]:
%time new_trans_proc, y_new, nas_new = proc_df(new_hist_trans, max_n_cat=7,ignore_flds=['purchase_date','card_id', 'merchant_id'])

CPU times: user 3.82 s, sys: 0 ns, total: 3.82 s
Wall time: 697 ms


In [14]:
hist_trans_proc.shape, new_trans_proc.shape

((29112361, 37), (1963031, 36))

In [15]:
hist_trans_proc.columns.values, new_trans_proc.columns.values

(array(['purchase_date', 'card_id', 'merchant_id', 'city_id', 'installments', 'merchant_category_id',
        'month_lag', 'purchase_amount', 'category_2', 'state_id', 'subsector_id', 'purchase_Year',
        'purchase_Month', 'purchase_Week', 'purchase_Day', 'purchase_Dayofweek', 'purchase_Dayofyear',
        'purchase_Is_month_end', 'purchase_Is_month_start', 'purchase_Is_quarter_end',
        'purchase_Is_quarter_start', 'purchase_Is_year_end', 'purchase_Is_year_start', 'purchase_Elapsed',
        'purchased_on_weekend', 'month_diff', 'category_2_na', 'authorized_flag_N', 'authorized_flag_Y',
        'authorized_flag_nan', 'category_1_N', 'category_1_Y', 'category_1_nan', 'category_3_A',
        'category_3_B', 'category_3_C', 'category_3_nan'], dtype=object),
 array(['purchase_date', 'card_id', 'merchant_id', 'city_id', 'installments', 'merchant_category_id',
        'month_lag', 'purchase_amount', 'category_2', 'state_id', 'subsector_id', 'purchase_Year',
        'purchase_Month',

In [16]:
import gc

In [17]:
gc.collect()

77

### Mark as categorical variables (not required)

In [72]:
def mark_categorical_cols_for_transactions(df):
    categorical_cols = ['authorized_flag', 'card_id', 'city_id',
       'merchant_category_id', 'merchant_id',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month']
    for c in categorical_cols:
        df[c] = df[c].astype('category').cat.as_ordered()
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [73]:
hist_trans_c, new_hist_trans_c = [mark_categorical_cols_for_transactions(df) for df in dfs]

In [74]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

In [75]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')

In [76]:
hist_trans = feather.read_dataframe('hist_trans')

In [77]:
new_hist_trans = feather.read_dataframe('new_hist_trans')

### One hot encoding categories (not required)

In [19]:
DataFrameSummary(new_hist_trans_c).summary()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff
count,1.96303e+06,NaN,NaN,1.96303e+06,1.96303e+06,NaN,NaN,NaN,1.96303e+06,1.96303e+06,...,1.96303e+06,NaN,NaN,NaN,NaN,NaN,NaN,1.96303e+06,1.96303e+06,1.96303e+06
mean,1,NaN,NaN,0.0321421,0.682964,NaN,NaN,NaN,1.47652,-0.550969,...,109.884,NaN,NaN,NaN,NaN,NaN,NaN,1.51935e+09,0.307438,11.772
std,0,NaN,NaN,0.176378,1.58407,NaN,NaN,NaN,0.499448,0.694004,...,75.1097,NaN,NaN,NaN,NaN,NaN,NaN,6.6753e+06,0.461433,2.55921
min,1,NaN,NaN,0,-1,NaN,NaN,NaN,1,-0.746893,...,1,NaN,NaN,NaN,NaN,NaN,NaN,1.48834e+09,0,10
25%,1,NaN,NaN,0,0,NaN,NaN,NaN,1,-0.716629,...,71,NaN,NaN,NaN,NaN,NaN,NaN,1.51967e+09,0,10
50%,1,NaN,NaN,0,1,NaN,NaN,NaN,1,-0.674841,...,90,NaN,NaN,NaN,NaN,NaN,NaN,1.52146e+09,0,11
75%,1,NaN,NaN,0,1,NaN,NaN,NaN,2,-0.581616,...,112,NaN,NaN,NaN,NaN,NaN,NaN,1.52322e+09,1,12
max,1,NaN,NaN,1,999,NaN,NaN,NaN,2,263.157,...,365,NaN,NaN,NaN,NaN,NaN,NaN,1.52513e+09,1,23
counts,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,...,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031
uniques,1,290001,308,2,15,3,314,226129,2,75190,...,365,2,2,2,2,2,2,1667025,2,14


In [20]:
def one_hot_encode_dfs(df):
    return pd.get_dummies(df, columns=['category_2', 'category_3'])

In [21]:
hist_trans_c.shape, new_hist_trans_c.shape

((29112361, 29), (1963031, 29))

In [22]:
hist_trans_c_e, new_hist_trans_c_e = [one_hot_encode_dfs(df) for df in dfs]

In [23]:
hist_trans_c_e.shape, new_hist_trans_c_e.shape

((29112361, 35), (1963031, 35))

In [24]:
new_hist_trans_c_e.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'state_id', 'subsector_id', 'purchase_Year',
       'purchase_Month', 'purchase_Week', 'purchase_Day', 'purchase_Dayofweek',
       'purchase_Dayofyear', 'purchase_Is_month_end',
       'purchase_Is_month_start', 'purchase_Is_quarter_end',
       'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed', 'purchased_on_weekend',
       'month_diff', 'category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C'],
      dtype='object')

In [25]:
## Not working
def aggregate_by_category(df):
    features = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C']
    cat_agg = {"purchase_amount": ["sum", "mean"], "installments": ["sum", "mean"]}
    other_df = pd.DataFrame()
    for fe in features:
        g = df.groupby(['card_id', fe]).agg(cat_agg)
        g.columns = ['_'+fe+'_'.join(c).strip() for c in g.columns.values]
        g.reset_index(inplace=True)
        if other_df.empty:
            other_df = g
        else:
            other_df = pd.merge(other_df, g, on='card_id', how='left')
    return other_df

### Aggregate by card_id

In [18]:
new_trans_proc['authorized_flag_N'] = 0

In [19]:
new_trans_proc.shape, hist_trans_proc.shape

((1963031, 37), (29112361, 37))

In [20]:
# DataFrameSummary(new_trans_proc).summary().T

In [21]:
# new_trans_proc.head().T

In [22]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['authorized_flag_Y'] = ['mean', 'sum']
    aggs['authorized_flag_N'] = ['mean', 'sum']
    aggs['purchase_amount'] = ['sum', 'max', 'min', 'mean', 'std', 'median']
    aggs['installments'] = ['sum', 'max', 'mean', 'std', 'median']
    aggs['month_lag'] = ['sum', 'max', 'min', 'mean', 'std', 'median']
    aggs['purchase_date'] = [np.min, np.max, np.ptp]
    aggs['card_id'] = ['size']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['category_1_N'] = ['sum', 'mean']
    aggs['category_1_Y'] = ['sum', 'mean']
    aggs['category_2'] = ['mean', 'sum']
    aggs['category_2_na'] = ['mean', 'sum']
    aggs['category_3_A'] = ['mean', 'sum']
    aggs['category_3_B' ] = ['mean', 'sum']
    aggs['category_3_C' ] = ['mean', 'sum']
    
    for col in ['category_2','category_2_na','category_3_nan', 'category_3_A', 'category_3_B', 'category_3_C']:
        df[col+'_mean2'] = df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean2'] = ['mean']    

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days    
    return new_df

In [23]:
dfs_e = [hist_trans_proc, new_trans_proc]

In [24]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in dfs_e]

CPU times: user 2min 56s, sys: 12.2 s, total: 3min 8s
Wall time: 1min 54s


### Add exta interpreted columns on aggregates

In [25]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_ptp']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_amax']).dt.days
    return df

In [26]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [27]:
hist_trans_agg.to_feather('hist_trans_agg_proc')
new_hist_trans_agg.to_feather('new_trans_agg_proc')

In [28]:
hist_trans_agg_proc = feather.read_dataframe('hist_trans_agg_proc')
new_trans_agg_proc = feather.read_dataframe('new_trans_agg_proc')

In [29]:
hist_trans_agg_proc.shape, new_trans_agg_proc.shape

((325540, 59), (290001, 59))

### Load test & train DFs

In [30]:
PATH = 'data/elo/'

In [31]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [32]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Merge train & test with new & old transactions history

In [33]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [34]:
hist_trans_agg.head().T

,0,1,2,3,4
card_id,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
transactions_count,149,123,66,216,144
city_id_nunique,4,18,3,10,9
merchant_category_id_nunique,18,29,19,48,36
merchant_id_nunique,29,65,28,119,73
state_id_nunique,3,6,2,4,7
subsector_id_nunique,13,17,12,24,21
authorized_flag_Y_mean,0.765101,0.97561,0.939394,0.875,0.951389
authorized_flag_Y_sum,114,120,62,189,137
authorized_flag_N_mean,0.234899,0.0243902,0.0606061,0.125,0.0486111


In [35]:
new_trans_agg_proc.head().T

,0,1,2,3,4
card_id,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
transactions_count,2,26,2,31,11
city_id_nunique,2,8,1,7,2
merchant_category_id_nunique,2,15,2,21,9
merchant_id_nunique,2,25,1,31,11
state_id_nunique,2,4,1,5,2
subsector_id_nunique,2,9,2,14,6
authorized_flag_Y_mean,1,1,1,1,1
authorized_flag_Y_sum,2,26,2,31,11
authorized_flag_N_mean,0,0,0,0,0


In [36]:
train_df = join_dfs(train, new_trans_agg_proc, left_on='card_id')

In [37]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [38]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [39]:
test_df.shape, train_df.shape

((123623, 134), (201917, 135))

In [40]:
train_df.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
first_active_monthYear,2017,2017,2016,2017,2017
first_active_monthMonth,6,1,8,9,11
first_active_monthWeek,22,52,31,35,44
first_active_monthDay,1,1,1,1,1


### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [43]:
display(train_df.columns.values)

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target',
       'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'first_active_monthElapsed',
       'transactions_count', 'city_id_nunique', 'merchant_category_id_nunique', 'merchant_id_nunique',
       'state_id_nunique', 'subsector_id_nunique', 'authorized_flag_Y_mean', 'authorized_flag_Y_sum',
       'authorized_flag_N_mean', 'authorized_flag_N_sum', 'purchase_amount_sum', 'purchase_amount_max',
       'purchase_amount_min', 'purchase_amount_mean', 'purchase_amount_std', 'purchase_amount_median',
       'installments_sum', 'installments_max', 'installme

In [44]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df["authorized_flag_Y_mean_total"]= df["authorized_flag_Y_mean_old"]+ df["authorized_flag_Y_mean"]
    df["authorized_flag_Y_sum_total"]= df["authorized_flag_Y_sum_old"]+ df["authorized_flag_Y_sum"]
    df["authorized_flag_N_mean_total"]= df["authorized_flag_N_mean_old"]+ df["authorized_flag_N_mean"]
    df["authorized_flag_N_sum_total"]= df["authorized_flag_N_sum_old"]+ df["authorized_flag_N_sum"]
    df["month_lag_sum_total"]= df["month_lag_sum_old"]+ df["month_lag_sum"]
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df["category_1_Y_sum_total"] = df["category_1_Y_sum_old"]+ df["category_1_Y_sum"]
    df["category_2_sum_total"]= df["category_2_sum_old"]+ df["category_2_sum"]
    df["category_3_A_total"]= df["category_3_A_sum"]+ df["category_3_A_sum_old"]
    df["category_3_B_total"]= df["category_3_B_sum"]+ df["category_3_B_sum_old"]
    df["category_3_C_total"]= df["category_3_C_sum"]+ df["category_3_C_sum_old"]
    df['hist_first_buy'] = (df['purchase_date_amin_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_amin'] - df['first_active_month']).dt.days

### Mark the outliers

In [45]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encode the outliers 

In [52]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [53]:
train_df.to_feather('train_df_proc')
test_df.to_feather('test_df_proc')